<a href="https://colab.research.google.com/github/sohelshekhatik1998/my-all-work/blob/main/Data_grokr__chess_data_analysis_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')
pd.set_option("display.max_rows", None, "display.max_columns", None)
spark = (SparkSession.builder.appName('<app_name>').getOrCreate())

In [3]:
import gdown

# a file
url = "https://drive.google.com/drive/folders/1QgWPHV_l25Ui9L7et8mkZohAOG59UTkQ"
output = "fcn8s_from_caffe.npz"
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/drive/folders/1QgWPHV_l25Ui9L7et8mkZohAOG59UTkQ
To: /content/fcn8s_from_caffe.npz
898kB [00:00, 45.7MB/s]


'fcn8s_from_caffe.npz'

In [4]:
# a folder
url = "https://drive.google.com/drive/folders/1QgWPHV_l25Ui9L7et8mkZohAOG59UTkQ"
gdown.download_folder(url, quiet=True, use_cookies=False)

['/content/chess/chess_schema.png',
 '/content/chess/chess_wc_history_game_info.csv',
 '/content/chess/chess_wc_history_moves.csv',
 '/content/chess/eco_codes.csv']

In [5]:
import cv2 as cv
from google.colab.patches import cv2_imshow
from skimage import io

In [6]:
#groupedDF = df.groupby('result',sort=False)['event',"white_first_name"].count()
#sortedDF=groupedDF.sort_values('black', ascending=False)['event']
#print(groupedDF)
#df2=df.loc[(df.result== '0-1') | (df.result=='1-0')] 
#df2.head()

In [7]:
game_info = pd.read_csv('/content/chess/chess_wc_history_game_info.csv')

game_info.head()


,game_id,game_order,event,site,date_played,round,white,black,result,white_elo,black_elo,white_title,black_title,winner,winner_elo,loser,loser_elo,winner_loser_elo_diff,eco,date_created,tournament_name
0,86e0b7f5-7b94-4ae3-97c8-317371622795,1,WCh 2021,Dubai UAE,2021.11.26,1.0,"Nepomniachtchi,I","Carlsen,M",1/2-1/2,2782.0,2855.0,NaN,NaN,draw,NaN,draw,NaN,0.0,C88,2022-07-22T22:33:50+0000,WorldChamp2021
1,dc4a10ab-54cf-49d0-b7ed-8c81a07e3e27,2,WCh 2021,Dubai UAE,2021.11.27,2.0,"Carlsen,M","Nepomniachtchi,I",1/2-1/2,2855.0,2782.0,NaN,NaN,draw,NaN,draw,NaN,0.0,E06,2022-07-22T22:33:50+0000,WorldChamp2021
2,f042ca37-8899-4887-87e9-e9d91c6e2d0e,3,WCh 2021,Dubai UAE,2021.11.28,3.0,"Nepomniachtchi,I","Carlsen,M",1/2-1/2,2782.0,2855.0,NaN,NaN,draw,NaN,draw,NaN,0.0,C88,2022-07-22T22:33:50+0000,WorldChamp2021
3,f70e4bbc-21e3-46f3-add0-7f0091822d5a,4,WCh 2021,Dubai UAE,2021.11.30,4.0,"Carlsen,M","Nepomniachtchi,I",1/2-1/2,2855.0,2782.0,NaN,NaN,draw,NaN,draw,NaN,0.0,C42,2022-07-22T22:33:50+0000,WorldChamp2021
4,c941c323-308a-4c86-9007-e1eb2a22cea6,5,WCh 2021,Dubai UAE,2021.12.01,5.0,"Nepomniachtchi,I","Carlsen,M",1/2-1/2,2782.0,2855.0,NaN,NaN,draw,NaN,draw,NaN,0.0,C88,2022-07-22T22:33:50+0000,WorldChamp2021


In [8]:
eco_df = pd.read_csv("/content/chess/eco_codes.csv")
eco_df.head()

,eco,eco_name,eco_example,eco_type,eco_group
0,A00,Uncommon Opening,"1 g4, a3, h3, etc.",A,Flank
1,A01,Nimzovich-Larsen Attack,1 b3,A,Flank
2,A02,Bird's Opening,1 f4,A,Flank
3,A03,Bird's Opening,1 f4 d5,A,Flank
4,A04,Reti Opening,1 Nf3,A,Flank


In [9]:
moves_df = pd.read_csv("/content/chess/chess_wc_history_moves.csv")
moves_df.head()

,game_id,move_no,move_no_pair,player,notation,move,from_square,to_square,piece,color,fen,is_check,is_check_mate,is_fifty_moves,is_fivefold_repetition,is_game_over,is_insufficient_material,white_count,black_count,white_pawn_count,black_pawn_count,white_queen_count,black_queen_count,white_bishop_count,black_bishop_count,white_knight_count,black_knight_count,white_rook_count,black_rook_count,captured_score_for_white,captured_score_for_black,fen_row1_white_count,fen_row2_white_count,fen_row3_white_count,fen_row4_white_count,fen_row5_white_count,fen_row6_white_count,fen_row7_white_count,fen_row8_white_count,fen_row1_white_value,fen_row2_white_value,fen_row3_white_value,fen_row4_white_value,fen_row5_white_value,fen_row6_white_value,fen_row7_white_value,fen_row8_white_value,fen_row1_black_count,fen_row2_black_count,fen_row3_black_count,fen_row4_black_count,fen_row5_black_count,fen_row6_black_count,fen_row7_black_count,fen_row8_black_count,fen_row1_black_value,fen_row2_black_value,fen_row3_black_value,fen_row4_black_value,fen_row5_black_value,fen_row6_black_value,fen_row7_black_value,fen_row8_black_value,move_sequence
0,86e0b7f5-7b94-4ae3-97c8-317371622795,1,1,"Nepomniachtchi,I",e4,e2e4,e2,e4,P,White,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR,0,0,0,0,0,0,16,16,8,8,1,1,2,2,2,2,2,2,0,0,8,7,0,1,0,0,0,0,31,7,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0,0,0,0,0,0,8,31,e4
1,86e0b7f5-7b94-4ae3-97c8-317371622795,2,1,"Carlsen,M",e5,e7e5,e7,e5,P,Black,rnbqkbnr/pppp1ppp/8/4p3/4P3/8/PPPP1PPP/RNBQKBNR,0,0,0,0,0,0,16,16,8,8,1,1,2,2,2,2,2,2,0,0,8,7,0,1,0,0,0,0,31,7,0,1,0,0,0,0,0,0,0,0,1,0,7,8,0,0,0,0,1,0,7,31,e4|e5
2,86e0b7f5-7b94-4ae3-97c8-317371622795,3,2,"Nepomniachtchi,I",Nf3,g1f3,g1,f3,N,White,rnbqkbnr/pppp1ppp/8/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R,0,0,0,0,0,0,16,16,8,8,1,1,2,2,2,2,2,2,0,0,7,7,1,1,0,0,0,0,28,7,3,1,0,0,0,0,0,0,0,0,1,0,7,8,0,0,0,0,1,0,7,31,e4|e5|Nf3
3,86e0b7f5-7b94-4ae3-97c8-317371622795,4,2,"Carlsen,M",Nc6,b8c6,b8,c6,N,Black,r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNB...,0,0,0,0,0,0,16,16,8,8,1,1,2,2,2,2,2,2,0,0,7,7,1,1,0,0,0,0,28,7,3,1,0,0,0,0,0,0,0,0,1,1,7,7,0,0,0,0,1,3,7,28,e4|e5|Nf3|Nc6
4,86e0b7f5-7b94-4ae3-97c8-317371622795,5,3,"Nepomniachtchi,I",Bb5,f1b5,f1,b5,B,White,r1bqkbnr/pppp1ppp/2n5/1B2p3/4P3/5N2/PPPP1PPP/R...,0,0,0,0,0,0,16,16,8,8,1,1,2,2,2,2,2,2,0,0,6,7,1,1,1,0,0,0,25,7,3,1,3,0,0,0,0,0,0,0,1,1,7,7,0,0,0,0,1,3,7,28,e4|e5|Nf3|Nc6|Bb5


In [10]:
game_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2938 entries, 0 to 2937
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   game_id                2938 non-null   object 
 1   game_order             2938 non-null   int64  
 2   event                  2938 non-null   object 
 3   site                   2938 non-null   object 
 4   date_played            2938 non-null   object 
 5   round                  2938 non-null   float64
 6   white                  2938 non-null   object 
 7   black                  2938 non-null   object 
 8   result                 2938 non-null   object 
 9   white_elo              2351 non-null   float64
 10  black_elo              2350 non-null   float64
 11  white_title            53 non-null     object 
 12  black_title            53 non-null     object 
 13  winner                 2938 non-null   object 
 14  winner_elo             1105 non-null   float64
 15  lose

In [11]:
game_info.tournament_name.nunique()

57

In [12]:
# convert to datetime

# df.date_played = pd.to_datetime(df.date_played)
game_info.date_created = pd.to_datetime(game_info.date_created)
# data played has missing values, dropping it is better
game_info = game_info.drop(columns=['date_played'])


In [13]:
game_info[['white_result', 'black_result']] = game_info['result'].str.split('-', expand=True)
game_info[['white_first_name','w_last_name']] = game_info['white'].str.split(',', expand=True)
game_info[['black_first_name', 'b_last_name']] = game_info['black'].str.split(",", expand=True)
game_info["white"] = game_info.white_first_name + "-" + game_info.white_result
game_info["black"] = game_info.black_first_name + "-" + game_info.black_result
game_info.head()

,game_id,game_order,event,site,round,white,black,result,white_elo,black_elo,white_title,black_title,winner,winner_elo,loser,loser_elo,winner_loser_elo_diff,eco,date_created,tournament_name,white_result,black_result,white_first_name,w_last_name,black_first_name,b_last_name
0,86e0b7f5-7b94-4ae3-97c8-317371622795,1,WCh 2021,Dubai UAE,1.0,Nepomniachtchi-1/2,Carlsen-1/2,1/2-1/2,2782.0,2855.0,NaN,NaN,draw,NaN,draw,NaN,0.0,C88,2022-07-22 22:33:50+00:00,WorldChamp2021,1/2,1/2,Nepomniachtchi,I,Carlsen,M
1,dc4a10ab-54cf-49d0-b7ed-8c81a07e3e27,2,WCh 2021,Dubai UAE,2.0,Carlsen-1/2,Nepomniachtchi-1/2,1/2-1/2,2855.0,2782.0,NaN,NaN,draw,NaN,draw,NaN,0.0,E06,2022-07-22 22:33:50+00:00,WorldChamp2021,1/2,1/2,Carlsen,M,Nepomniachtchi,I
2,f042ca37-8899-4887-87e9-e9d91c6e2d0e,3,WCh 2021,Dubai UAE,3.0,Nepomniachtchi-1/2,Carlsen-1/2,1/2-1/2,2782.0,2855.0,NaN,NaN,draw,NaN,draw,NaN,0.0,C88,2022-07-22 22:33:50+00:00,WorldChamp2021,1/2,1/2,Nepomniachtchi,I,Carlsen,M
3,f70e4bbc-21e3-46f3-add0-7f0091822d5a,4,WCh 2021,Dubai UAE,4.0,Carlsen-1/2,Nepomniachtchi-1/2,1/2-1/2,2855.0,2782.0,NaN,NaN,draw,NaN,draw,NaN,0.0,C42,2022-07-22 22:33:50+00:00,WorldChamp2021,1/2,1/2,Carlsen,M,Nepomniachtchi,I
4,c941c323-308a-4c86-9007-e1eb2a22cea6,5,WCh 2021,Dubai UAE,5.0,Nepomniachtchi-1/2,Carlsen-1/2,1/2-1/2,2782.0,2855.0,NaN,NaN,draw,NaN,draw,NaN,0.0,C88,2022-07-22 22:33:50+00:00,WorldChamp2021,1/2,1/2,Nepomniachtchi,I,Carlsen,M


### Marge dataset

In [14]:
# merge 1
df = pd.merge(game_info, moves_df, how='left', on='game_id')

# merge2
df = pd.merge(df, eco_df, how='left', on='eco')
df.head()

,game_id,game_order,event,site,round,white,black,result,white_elo,black_elo,white_title,black_title,winner,winner_elo,loser,loser_elo,winner_loser_elo_diff,eco,date_created,tournament_name,white_result,black_result,white_first_name,w_last_name,black_first_name,b_last_name,move_no,move_no_pair,player,notation,move,from_square,to_square,piece,color,fen,is_check,is_check_mate,is_fifty_moves,is_fivefold_repetition,is_game_over,is_insufficient_material,white_count,black_count,white_pawn_count,black_pawn_count,white_queen_count,black_queen_count,white_bishop_count,black_bishop_count,white_knight_count,black_knight_count,white_rook_count,black_rook_count,captured_score_for_white,captured_score_for_black,fen_row1_white_count,fen_row2_white_count,fen_row3_white_count,fen_row4_white_count,fen_row5_white_count,fen_row6_white_count,fen_row7_white_count,fen_row8_white_count,fen_row1_white_value,fen_row2_white_value,fen_row3_white_value,fen_row4_white_value,fen_row5_white_value,fen_row6_white_value,fen_row7_white_value,fen_row8_white_value,fen_row1_black_count,fen_row2_black_count,fen_row3_black_count,fen_row4_black_count,fen_row5_black_count,fen_row6_black_count,fen_row7_black_count,fen_row8_black_count,fen_row1_black_value,fen_row2_black_value,fen_row3_black_value,fen_row4_black_value,fen_row5_black_value,fen_row6_black_value,fen_row7_black_value,fen_row8_black_value,move_sequence,eco_name,eco_example,eco_type,eco_group
0,86e0b7f5-7b94-4ae3-97c8-317371622795,1,WCh 2021,Dubai UAE,1.0,Nepomniachtchi-1/2,Carlsen-1/2,1/2-1/2,2782.0,2855.0,NaN,NaN,draw,NaN,draw,NaN,0.0,C88,2022-07-22 22:33:50+00:00,WorldChamp2021,1/2,1/2,Nepomniachtchi,I,Carlsen,M,1.0,1.0,"Nepomniachtchi,I",e4,e2e4,e2,e4,P,White,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR,0.0,0.0,0.0,0.0,0.0,0.0,16.0,16.0,8.0,8.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,8.0,7.0,0.0,1.0,0.0,0.0,0.0,0.0,31.0,7.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,31.0,e4,Ruy Lopez,1 e4 e5 2 Nf3 Nc6 3 Bb5 a6 4 Ba4 Nf6 5 O-O Be7...,C,Open (inc French)
1,86e0b7f5-7b94-4ae3-97c8-317371622795,1,WCh 2021,Dubai UAE,1.0,Nepomniachtchi-1/2,Carlsen-1/2,1/2-1/2,2782.0,2855.0,NaN,NaN,draw,NaN,draw,NaN,0.0,C88,2022-07-22 22:33:50+00:00,WorldChamp2021,1/2,1/2,Nepomniachtchi,I,Carlsen,M,2.0,1.0,"Carlsen,M",e5,e7e5,e7,e5,P,Black,rnbqkbnr/pppp1ppp/8/4p3/4P3/8/PPPP1PPP/RNBQKBNR,0.0,0.0,0.0,0.0,0.0,0.0,16.0,16.0,8.0,8.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,8.0,7.0,0.0,1.0,0.0,0.0,0.0,0.0,31.0,7.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,7.0,8.0,0.0,0.0,0.0,0.0,1.0,0.0,7.0,31.0,e4|e5,Ruy Lopez,1 e4 e5 2 Nf3 Nc6 3 Bb5 a6 4 Ba4 Nf6 5 O-O Be7...,C,Open (inc French)
2,86e0b7f5-7b94-4ae3-97c8-317371622795,1,WCh 2021,Dubai UAE,1.0,Nepomniachtchi-1/2,Carlsen-1/2,1/2-1/2,2782.0,2855.0,NaN,NaN,draw,NaN,draw,NaN,0.0,C88,2022-07-22 22:33:50+00:00,WorldChamp2021,1/2,1/2,Nepomniachtchi,I,Carlsen,M,3.0,2.0,"Nepomniachtchi,I",Nf3,g1f3,g1,f3,N,White,rnbqkbnr/pppp1ppp/8/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R,0.0,0.0,0.0,0.0,0.0,0.0,16.0,16.0,8.0,8.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,7.0,7.0,1.0,1.0,0.0,0.0,0.0,0.0,28.0,7.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,7.0,8.0,0.0,0.0,0.0,0.0,1.0,0.0,7.0,31.0,e4|e5|Nf3,Ruy Lopez,1 e4 e5 2 Nf3 Nc6 3 Bb5 a6 4 Ba4 Nf6 5 O-O Be7...,C,Open (inc French)
3,86e0b7f5-7b94-4ae3-97c8-317371622795,1,WCh 2021,Dubai UAE,1.0,Nepomniachtchi-1/2,Carlsen-1/2,1/2-1/2,2782.0,2855.0,NaN,NaN,draw,NaN,draw,NaN,0.0,C88,2022-07-22 22:33:50+00:00,WorldChamp2021,1/2,1/2,Nepomniachtchi,I,Carlsen,M,4.0,2.0,"Carlsen,M",Nc6,b8c6,b8,c6,N,Black,r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNB...,0.0,0.0,0.0,0.0,0.0,0.0,16.0,16.0,8.0,8.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,7.0,7.0,1.0,1.0,0.0,0.0,0.0,0.0,28.0,7.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,7.0,7.0,0.0,0.0,0.0,0.0,1.0,3.0,7.0,28.0,e4|e5|Nf3|Nc6,Ruy Lopez,1 e4 e5 2 Nf3 Nc6 3 Bb5 a6 4 Ba4 Nf6 5 O-O Be7...,C,Open (inc French)
4,86e0b7f5-7b94-4ae3-97c8-317371622795,1,WCh 2021,Dubai UAE,1.0,Nepomniachtchi-1/

In [15]:
df[['player', 'fault']] = df['player'].str.split(",", expand=True)
df.head()

,game_id,game_order,event,site,round,white,black,result,white_elo,black_elo,white_title,black_title,winner,winner_elo,loser,loser_elo,winner_loser_elo_diff,eco,date_created,tournament_name,white_result,black_result,white_first_name,w_last_name,black_first_name,b_last_name,move_no,move_no_pair,player,notation,move,from_square,to_square,piece,color,fen,is_check,is_check_mate,is_fifty_moves,is_fivefold_repetition,is_game_over,is_insufficient_material,white_count,black_count,white_pawn_count,black_pawn_count,white_queen_count,black_queen_count,white_bishop_count,black_bishop_count,white_knight_count,black_knight_count,white_rook_count,black_rook_count,captured_score_for_white,captured_score_for_black,fen_row1_white_count,fen_row2_white_count,fen_row3_white_count,fen_row4_white_count,fen_row5_white_count,fen_row6_white_count,fen_row7_white_count,fen_row8_white_count,fen_row1_white_value,fen_row2_white_value,fen_row3_white_value,fen_row4_white_value,fen_row5_white_value,fen_row6_white_value,fen_row7_white_value,fen_row8_white_value,fen_row1_black_count,fen_row2_black_count,fen_row3_black_count,fen_row4_black_count,fen_row5_black_count,fen_row6_black_count,fen_row7_black_count,fen_row8_black_count,fen_row1_black_value,fen_row2_black_value,fen_row3_black_value,fen_row4_black_value,fen_row5_black_value,fen_row6_black_value,fen_row7_black_value,fen_row8_black_value,move_sequence,eco_name,eco_example,eco_type,eco_group,fault
0,86e0b7f5-7b94-4ae3-97c8-317371622795,1,WCh 2021,Dubai UAE,1.0,Nepomniachtchi-1/2,Carlsen-1/2,1/2-1/2,2782.0,2855.0,NaN,NaN,draw,NaN,draw,NaN,0.0,C88,2022-07-22 22:33:50+00:00,WorldChamp2021,1/2,1/2,Nepomniachtchi,I,Carlsen,M,1.0,1.0,Nepomniachtchi,e4,e2e4,e2,e4,P,White,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR,0.0,0.0,0.0,0.0,0.0,0.0,16.0,16.0,8.0,8.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,8.0,7.0,0.0,1.0,0.0,0.0,0.0,0.0,31.0,7.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,31.0,e4,Ruy Lopez,1 e4 e5 2 Nf3 Nc6 3 Bb5 a6 4 Ba4 Nf6 5 O-O Be7...,C,Open (inc French),I
1,86e0b7f5-7b94-4ae3-97c8-317371622795,1,WCh 2021,Dubai UAE,1.0,Nepomniachtchi-1/2,Carlsen-1/2,1/2-1/2,2782.0,2855.0,NaN,NaN,draw,NaN,draw,NaN,0.0,C88,2022-07-22 22:33:50+00:00,WorldChamp2021,1/2,1/2,Nepomniachtchi,I,Carlsen,M,2.0,1.0,Carlsen,e5,e7e5,e7,e5,P,Black,rnbqkbnr/pppp1ppp/8/4p3/4P3/8/PPPP1PPP/RNBQKBNR,0.0,0.0,0.0,0.0,0.0,0.0,16.0,16.0,8.0,8.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,8.0,7.0,0.0,1.0,0.0,0.0,0.0,0.0,31.0,7.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,7.0,8.0,0.0,0.0,0.0,0.0,1.0,0.0,7.0,31.0,e4|e5,Ruy Lopez,1 e4 e5 2 Nf3 Nc6 3 Bb5 a6 4 Ba4 Nf6 5 O-O Be7...,C,Open (inc French),M
2,86e0b7f5-7b94-4ae3-97c8-317371622795,1,WCh 2021,Dubai UAE,1.0,Nepomniachtchi-1/2,Carlsen-1/2,1/2-1/2,2782.0,2855.0,NaN,NaN,draw,NaN,draw,NaN,0.0,C88,2022-07-22 22:33:50+00:00,WorldChamp2021,1/2,1/2,Nepomniachtchi,I,Carlsen,M,3.0,2.0,Nepomniachtchi,Nf3,g1f3,g1,f3,N,White,rnbqkbnr/pppp1ppp/8/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R,0.0,0.0,0.0,0.0,0.0,0.0,16.0,16.0,8.0,8.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,7.0,7.0,1.0,1.0,0.0,0.0,0.0,0.0,28.0,7.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,7.0,8.0,0.0,0.0,0.0,0.0,1.0,0.0,7.0,31.0,e4|e5|Nf3,Ruy Lopez,1 e4 e5 2 Nf3 Nc6 3 Bb5 a6 4 Ba4 Nf6 5 O-O Be7...,C,Open (inc French),I
3,86e0b7f5-7b94-4ae3-97c8-317371622795,1,WCh 2021,Dubai UAE,1.0,Nepomniachtchi-1/2,Carlsen-1/2,1/2-1/2,2782.0,2855.0,NaN,NaN,draw,NaN,draw,NaN,0.0,C88,2022-07-22 22:33:50+00:00,WorldChamp2021,1/2,1/2,Nepomniachtchi,I,Carlsen,M,4.0,2.0,Carlsen,Nc6,b8c6,b8,c6,N,Black,r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNB...,0.0,0.0,0.0,0.0,0.0,0.0,16.0,16.0,8.0,8.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,7.0,7.0,1.0,1.0,0.0,0.0,0.0,0.0,28.0,7.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,7.0,7.0,0.0,0.0,0.0,0.0,1.0,3.0,7.0,28.0,e4|e5|Nf3|Nc6,Ruy Lopez,1 e4 e5 2 Nf3 Nc6 3 Bb5 a6 4 Ba4 Nf6 5 O-O Be7...,C,Open (inc French),M
4,86e0b7f5-7b94-4ae3-97c8-317371622795,1,WCh 2021,Dubai UAE,1.0,Nepomniachtchi-1/2,

In [16]:
df.drop(['result'], axis=1, inplace=True)
df.head()

,game_id,game_order,event,site,round,white,black,white_elo,black_elo,white_title,black_title,winner,winner_elo,loser,loser_elo,winner_loser_elo_diff,eco,date_created,tournament_name,white_result,black_result,white_first_name,w_last_name,black_first_name,b_last_name,move_no,move_no_pair,player,notation,move,from_square,to_square,piece,color,fen,is_check,is_check_mate,is_fifty_moves,is_fivefold_repetition,is_game_over,is_insufficient_material,white_count,black_count,white_pawn_count,black_pawn_count,white_queen_count,black_queen_count,white_bishop_count,black_bishop_count,white_knight_count,black_knight_count,white_rook_count,black_rook_count,captured_score_for_white,captured_score_for_black,fen_row1_white_count,fen_row2_white_count,fen_row3_white_count,fen_row4_white_count,fen_row5_white_count,fen_row6_white_count,fen_row7_white_count,fen_row8_white_count,fen_row1_white_value,fen_row2_white_value,fen_row3_white_value,fen_row4_white_value,fen_row5_white_value,fen_row6_white_value,fen_row7_white_value,fen_row8_white_value,fen_row1_black_count,fen_row2_black_count,fen_row3_black_count,fen_row4_black_count,fen_row5_black_count,fen_row6_black_count,fen_row7_black_count,fen_row8_black_count,fen_row1_black_value,fen_row2_black_value,fen_row3_black_value,fen_row4_black_value,fen_row5_black_value,fen_row6_black_value,fen_row7_black_value,fen_row8_black_value,move_sequence,eco_name,eco_example,eco_type,eco_group,fault
0,86e0b7f5-7b94-4ae3-97c8-317371622795,1,WCh 2021,Dubai UAE,1.0,Nepomniachtchi-1/2,Carlsen-1/2,2782.0,2855.0,NaN,NaN,draw,NaN,draw,NaN,0.0,C88,2022-07-22 22:33:50+00:00,WorldChamp2021,1/2,1/2,Nepomniachtchi,I,Carlsen,M,1.0,1.0,Nepomniachtchi,e4,e2e4,e2,e4,P,White,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR,0.0,0.0,0.0,0.0,0.0,0.0,16.0,16.0,8.0,8.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,8.0,7.0,0.0,1.0,0.0,0.0,0.0,0.0,31.0,7.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,31.0,e4,Ruy Lopez,1 e4 e5 2 Nf3 Nc6 3 Bb5 a6 4 Ba4 Nf6 5 O-O Be7...,C,Open (inc French),I
1,86e0b7f5-7b94-4ae3-97c8-317371622795,1,WCh 2021,Dubai UAE,1.0,Nepomniachtchi-1/2,Carlsen-1/2,2782.0,2855.0,NaN,NaN,draw,NaN,draw,NaN,0.0,C88,2022-07-22 22:33:50+00:00,WorldChamp2021,1/2,1/2,Nepomniachtchi,I,Carlsen,M,2.0,1.0,Carlsen,e5,e7e5,e7,e5,P,Black,rnbqkbnr/pppp1ppp/8/4p3/4P3/8/PPPP1PPP/RNBQKBNR,0.0,0.0,0.0,0.0,0.0,0.0,16.0,16.0,8.0,8.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,8.0,7.0,0.0,1.0,0.0,0.0,0.0,0.0,31.0,7.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,7.0,8.0,0.0,0.0,0.0,0.0,1.0,0.0,7.0,31.0,e4|e5,Ruy Lopez,1 e4 e5 2 Nf3 Nc6 3 Bb5 a6 4 Ba4 Nf6 5 O-O Be7...,C,Open (inc French),M
2,86e0b7f5-7b94-4ae3-97c8-317371622795,1,WCh 2021,Dubai UAE,1.0,Nepomniachtchi-1/2,Carlsen-1/2,2782.0,2855.0,NaN,NaN,draw,NaN,draw,NaN,0.0,C88,2022-07-22 22:33:50+00:00,WorldChamp2021,1/2,1/2,Nepomniachtchi,I,Carlsen,M,3.0,2.0,Nepomniachtchi,Nf3,g1f3,g1,f3,N,White,rnbqkbnr/pppp1ppp/8/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R,0.0,0.0,0.0,0.0,0.0,0.0,16.0,16.0,8.0,8.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,7.0,7.0,1.0,1.0,0.0,0.0,0.0,0.0,28.0,7.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,7.0,8.0,0.0,0.0,0.0,0.0,1.0,0.0,7.0,31.0,e4|e5|Nf3,Ruy Lopez,1 e4 e5 2 Nf3 Nc6 3 Bb5 a6 4 Ba4 Nf6 5 O-O Be7...,C,Open (inc French),I
3,86e0b7f5-7b94-4ae3-97c8-317371622795,1,WCh 2021,Dubai UAE,1.0,Nepomniachtchi-1/2,Carlsen-1/2,2782.0,2855.0,NaN,NaN,draw,NaN,draw,NaN,0.0,C88,2022-07-22 22:33:50+00:00,WorldChamp2021,1/2,1/2,Nepomniachtchi,I,Carlsen,M,4.0,2.0,Carlsen,Nc6,b8c6,b8,c6,N,Black,r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNB...,0.0,0.0,0.0,0.0,0.0,0.0,16.0,16.0,8.0,8.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,7.0,7.0,1.0,1.0,0.0,0.0,0.0,0.0,28.0,7.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,7.0,7.0,0.0,0.0,0.0,0.0,1.0,3.0,7.0,28.0,e4|e5|Nf3|Nc6,Ruy Lopez,1 e4 e5 2 Nf3 Nc6 3 Bb5 a6 4 Ba4 Nf6 5 O-O Be7...,C,Open (inc French),M
4,86e0b7f5-7b94-4ae3-97c8-317371622795,1,WCh 2021,Dubai UAE,1.0,Nepomniachtchi-1/2,Carlsen-1/2,2782.0,2855.0,NaN,NaN,draw,

In [17]:
df.drop(['fault','w_last_name','b_last_name'], axis=1, inplace=True)
df.head()

,game_id,game_order,event,site,round,white,black,white_elo,black_elo,white_title,black_title,winner,winner_elo,loser,loser_elo,winner_loser_elo_diff,eco,date_created,tournament_name,white_result,black_result,white_first_name,black_first_name,move_no,move_no_pair,player,notation,move,from_square,to_square,piece,color,fen,is_check,is_check_mate,is_fifty_moves,is_fivefold_repetition,is_game_over,is_insufficient_material,white_count,black_count,white_pawn_count,black_pawn_count,white_queen_count,black_queen_count,white_bishop_count,black_bishop_count,white_knight_count,black_knight_count,white_rook_count,black_rook_count,captured_score_for_white,captured_score_for_black,fen_row1_white_count,fen_row2_white_count,fen_row3_white_count,fen_row4_white_count,fen_row5_white_count,fen_row6_white_count,fen_row7_white_count,fen_row8_white_count,fen_row1_white_value,fen_row2_white_value,fen_row3_white_value,fen_row4_white_value,fen_row5_white_value,fen_row6_white_value,fen_row7_white_value,fen_row8_white_value,fen_row1_black_count,fen_row2_black_count,fen_row3_black_count,fen_row4_black_count,fen_row5_black_count,fen_row6_black_count,fen_row7_black_count,fen_row8_black_count,fen_row1_black_value,fen_row2_black_value,fen_row3_black_value,fen_row4_black_value,fen_row5_black_value,fen_row6_black_value,fen_row7_black_value,fen_row8_black_value,move_sequence,eco_name,eco_example,eco_type,eco_group
0,86e0b7f5-7b94-4ae3-97c8-317371622795,1,WCh 2021,Dubai UAE,1.0,Nepomniachtchi-1/2,Carlsen-1/2,2782.0,2855.0,NaN,NaN,draw,NaN,draw,NaN,0.0,C88,2022-07-22 22:33:50+00:00,WorldChamp2021,1/2,1/2,Nepomniachtchi,Carlsen,1.0,1.0,Nepomniachtchi,e4,e2e4,e2,e4,P,White,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR,0.0,0.0,0.0,0.0,0.0,0.0,16.0,16.0,8.0,8.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,8.0,7.0,0.0,1.0,0.0,0.0,0.0,0.0,31.0,7.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,31.0,e4,Ruy Lopez,1 e4 e5 2 Nf3 Nc6 3 Bb5 a6 4 Ba4 Nf6 5 O-O Be7...,C,Open (inc French)
1,86e0b7f5-7b94-4ae3-97c8-317371622795,1,WCh 2021,Dubai UAE,1.0,Nepomniachtchi-1/2,Carlsen-1/2,2782.0,2855.0,NaN,NaN,draw,NaN,draw,NaN,0.0,C88,2022-07-22 22:33:50+00:00,WorldChamp2021,1/2,1/2,Nepomniachtchi,Carlsen,2.0,1.0,Carlsen,e5,e7e5,e7,e5,P,Black,rnbqkbnr/pppp1ppp/8/4p3/4P3/8/PPPP1PPP/RNBQKBNR,0.0,0.0,0.0,0.0,0.0,0.0,16.0,16.0,8.0,8.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,8.0,7.0,0.0,1.0,0.0,0.0,0.0,0.0,31.0,7.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,7.0,8.0,0.0,0.0,0.0,0.0,1.0,0.0,7.0,31.0,e4|e5,Ruy Lopez,1 e4 e5 2 Nf3 Nc6 3 Bb5 a6 4 Ba4 Nf6 5 O-O Be7...,C,Open (inc French)
2,86e0b7f5-7b94-4ae3-97c8-317371622795,1,WCh 2021,Dubai UAE,1.0,Nepomniachtchi-1/2,Carlsen-1/2,2782.0,2855.0,NaN,NaN,draw,NaN,draw,NaN,0.0,C88,2022-07-22 22:33:50+00:00,WorldChamp2021,1/2,1/2,Nepomniachtchi,Carlsen,3.0,2.0,Nepomniachtchi,Nf3,g1f3,g1,f3,N,White,rnbqkbnr/pppp1ppp/8/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R,0.0,0.0,0.0,0.0,0.0,0.0,16.0,16.0,8.0,8.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,7.0,7.0,1.0,1.0,0.0,0.0,0.0,0.0,28.0,7.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,7.0,8.0,0.0,0.0,0.0,0.0,1.0,0.0,7.0,31.0,e4|e5|Nf3,Ruy Lopez,1 e4 e5 2 Nf3 Nc6 3 Bb5 a6 4 Ba4 Nf6 5 O-O Be7...,C,Open (inc French)
3,86e0b7f5-7b94-4ae3-97c8-317371622795,1,WCh 2021,Dubai UAE,1.0,Nepomniachtchi-1/2,Carlsen-1/2,2782.0,2855.0,NaN,NaN,draw,NaN,draw,NaN,0.0,C88,2022-07-22 22:33:50+00:00,WorldChamp2021,1/2,1/2,Nepomniachtchi,Carlsen,4.0,2.0,Carlsen,Nc6,b8c6,b8,c6,N,Black,r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNB...,0.0,0.0,0.0,0.0,0.0,0.0,16.0,16.0,8.0,8.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,7.0,7.0,1.0,1.0,0.0,0.0,0.0,0.0,28.0,7.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,7.0,7.0,0.0,0.0,0.0,0.0,1.0,3.0,7.0,28.0,e4|e5|Nf3|Nc6,Ruy Lopez,1 e4 e5 2 Nf3 Nc6 3 Bb5 a6 4 Ba4 Nf6 5 O-O Be7...,C,Open (inc French)
4,86e0b7f5-7b94-4ae3-97c8-317371622795,1,WCh 2021,Dubai UAE,1.0,Nepomniachtchi-1/2,Carlsen-1/2,2782.0,2855.0,NaN,NaN,draw,NaN,draw,NaN,0.0,C88,2022-07-22 22:33:50+00:00,WorldCh

### Q.1. List of winners of each world champions Trophy

In [18]:

winners_per_tournament = pd.DataFrame(game_info.groupby(['tournament_name','winner']).winner.count()).rename(columns={'winner':'score'}).reset_index()
winners_per_tournament = winners_per_tournament[winners_per_tournament.winner != 'draw']
winners_per_tournament = winners_per_tournament.loc[winners_per_tournament.groupby(['tournament_name'])['score'].idxmax()]
winners_per_tournament.head()

,tournament_name,winner,score
0,FideChamp1993,"Karpov, Anatoly",6
4,FideChamp1996,"Karpov, Anatoly",6
12,FideChamp1998,"Anand, Viswanathan",10
97,FideChamp1999,"Khalifman, Alexander",12
142,FideChamp2000,"Anand,V",8


### Q. List of Players with number of times they have won Tournament in descending order(Max to min).

Result attributes: player_name, number_of_wins

In [19]:
winners_per_tournament.sort_values('score', ascending=False)[['winner', 'score']]
winners_per_tournament.head()

,tournament_name,winner,score
0,FideChamp1993,"Karpov, Anatoly",6
4,FideChamp1996,"Karpov, Anatoly",6
12,FideChamp1998,"Anand, Viswanathan",10
97,FideChamp1999,"Khalifman, Alexander",12
142,FideChamp2000,"Anand,V",8


### Q.3. Most and Least Popular eco move in world championship history.

In [20]:
eco_moves = pd.DataFrame(game_info.groupby('eco').eco.count()).rename(columns={'eco':'count'}).reset_index()

# most popular 
print("Most popular moves:")
print(eco_moves.sort_values('count', ascending=False).head(1))
print()

# least popular
print("Least popular moves")
print(eco_moves.sort_values('count', ascending=True).head(1))

Most popular moves:
     eco  count
193  C42     64

Least popular moves
     eco  count
411  E99      1


### Q.4. Find the eco move with most winnings.

In [21]:
eco_move_win = pd.DataFrame(game_info[game_info.winner != 'draw'].groupby(['eco']).winner.count()).rename(columns={'winner':'wins'})
eco_move_win = pd.merge(eco_move_win, eco_df[['eco','eco_name']], how='left', on='eco')
eco_move_win.max()

eco                 E99
wins                 25
eco_name    Vienna Game
dtype: object

### Q.5. Longest and shortest game ever played in a world championship in terms of move.

In [22]:
game_time = pd.DataFrame(df.groupby(['game_id', 'event', 'tournament_name']).move_no.max()).rename(columns={'move_no':'number_of_moves'}).reset_index()
print("Longest game ever played: ")
game_time.max()

Longest game ever played: 


game_id            ffdfe902-b93f-4b5d-826b-cab5c25a9d2f
event                            World Championship 9th
tournament_name                          WorldChamp2021
number_of_moves                                   291.0
dtype: object

In [23]:
print("Shortest game ever played: ")
game_time.min()

Shortest game ever played: 


game_id            0028cc2e-5aaa-4645-a8e9-4d6b51c8f9db
event                                    Braingames WCC
tournament_name                           FideChamp1993
number_of_moves                                     1.0
dtype: object

### Q.6. Shortest and Longest Draw game ever played

In [24]:
draw_games = df[df.winner=='draw']
draw_games_time = pd.DataFrame(draw_games.groupby(['game_id','event', 'tournament_name']).move_no.count()).rename(columns={'move_no':'number_of_moves'}).reset_index()

print("Longest draw game ever played: ")
print(draw_games_time.max())
print()

print("Shortest draw game ever played: ")
print(draw_games_time.min())


Longest draw game ever played: 
game_id            ffb635d0-7e47-49b1-a83b-0a4c7250d30b
event                            World Championship 9th
tournament_name                          WorldChamp2021
number_of_moves                                     258
dtype: object

Shortest draw game ever played: 
game_id            0028cc2e-5aaa-4645-a8e9-4d6b51c8f9db
event                                    Braingames WCC
tournament_name                           FideChamp1993
number_of_moves                                      12
dtype: object


### Q.7. Most and Least rated Player

In [25]:
# Most rated player:
print("Most rated player: ")
winner_elo = pd.DataFrame(game_info[game_info.winner != 'draw'].groupby(['winner']).winner_elo.sum()).reset_index().rename(columns={'winner':'player_name', 'winner_elo':'elo'})
winner_elo.max()

Most rated player: 


player_name    Zvjaginsev,V
elo                128175.0
dtype: object

In [26]:
# Least rated player
print("Least rated player: ")
least_rated_player = pd.DataFrame(game_info[game_info.winner != 'draw'].groupby(['loser']).loser_elo.sum()).reset_index().rename(columns={'loser':'player_name', 'loser_elo':'elo'})
least_rated_player.min()

Least rated player: 


player_name    Abulhul,T
elo                  0.0
dtype: object

### Q.8. 3rd Last Player with most loss

In [27]:
loss_df = game_info[game_info.winner !='draw'].groupby(['loser']).loser.count().sort_values()
loss_df[2:3]

loser
Smirin, Ilia    1
Name: loser, dtype: int64

### Q.9. How many times players with low rating won matches with their total win Count.

In [28]:
# Number of wins
win_count = pd.DataFrame(game_info[game_info.winner!='draw'].groupby(['winner']).winner.count().sort_values()).rename(columns={'winner':'win_count'}).reset_index().rename(columns={'winner':'player_name'})

# merge win count with win elo
win_elo_df = pd.merge(winner_elo, win_count, how='left', on='player_name')

# win of lowest rating players
win_elo_df.loc[win_elo_df.elo == win_elo_df.elo.min()]

,player_name,elo,win_count
8,"Alekhine, Alexander",0.0,43
37,"Bogoljubow, Efim",0.0,8
40,"Botvinnik, Mikhail",0.0,46
41,"Bronstein, David I",0.0,5
44,"Capablanca, Jose Raul",0.0,7
49,"Chigorin, Mikhail",0.0,14
64,"Euwe, Max",0.0,14
86,"Gunsberg, Isidor",0.0,4
101,"Janowsky, Dawid Markelowicz",0.0,1
108,"Karlov,A",0.0,1


### Q.10. Move sequence for each player in a match

In [29]:
#move_sequence_df = pd.pivot_table(data=moves_df, index=['game_id','player'], columns=['move_sequence'], values=['move_no'], aggfunc='max')
# move_sequence_df.head()

### Q.11. Total number of games where losing player has more captured score than winning player.
Hint: Captured score is cumulative, i.e., for 3rd capture it will have score for 1, 2, and 3rd.
Result attributes: total_number_of_games Final result will have only one row

In [30]:
game_info.head()

,game_id,game_order,event,site,round,white,black,result,white_elo,black_elo,white_title,black_title,winner,winner_elo,loser,loser_elo,winner_loser_elo_diff,eco,date_created,tournament_name,white_result,black_result,white_first_name,w_last_name,black_first_name,b_last_name
0,86e0b7f5-7b94-4ae3-97c8-317371622795,1,WCh 2021,Dubai UAE,1.0,Nepomniachtchi-1/2,Carlsen-1/2,1/2-1/2,2782.0,2855.0,NaN,NaN,draw,NaN,draw,NaN,0.0,C88,2022-07-22 22:33:50+00:00,WorldChamp2021,1/2,1/2,Nepomniachtchi,I,Carlsen,M
1,dc4a10ab-54cf-49d0-b7ed-8c81a07e3e27,2,WCh 2021,Dubai UAE,2.0,Carlsen-1/2,Nepomniachtchi-1/2,1/2-1/2,2855.0,2782.0,NaN,NaN,draw,NaN,draw,NaN,0.0,E06,2022-07-22 22:33:50+00:00,WorldChamp2021,1/2,1/2,Carlsen,M,Nepomniachtchi,I
2,f042ca37-8899-4887-87e9-e9d91c6e2d0e,3,WCh 2021,Dubai UAE,3.0,Nepomniachtchi-1/2,Carlsen-1/2,1/2-1/2,2782.0,2855.0,NaN,NaN,draw,NaN,draw,NaN,0.0,C88,2022-07-22 22:33:50+00:00,WorldChamp2021,1/2,1/2,Nepomniachtchi,I,Carlsen,M
3,f70e4bbc-21e3-46f3-add0-7f0091822d5a,4,WCh 2021,Dubai UAE,4.0,Carlsen-1/2,Nepomniachtchi-1/2,1/2-1/2,2855.0,2782.0,NaN,NaN,draw,NaN,draw,NaN,0.0,C42,2022-07-22 22:33:50+00:00,WorldChamp2021,1/2,1/2,Carlsen,M,Nepomniachtchi,I
4,c941c323-308a-4c86-9007-e1eb2a22cea6,5,WCh 2021,Dubai UAE,5.0,Nepomniachtchi-1/2,Carlsen-1/2,1/2-1/2,2782.0,2855.0,NaN,NaN,draw,NaN,draw,NaN,0.0,C88,2022-07-22 22:33:50+00:00,WorldChamp2021,1/2,1/2,Nepomniachtchi,I,Carlsen,M


In [31]:
def result(row):
    if len(row) == 1:
        return int(row)
    else:
        x,y = row.split('/')
        res = int(x)/int(y)
        return res


game_info.black_result = game_info.black_result.apply(result)
game_info.white_result = game_info.white_result.apply(result)

In [32]:
white_result = game_info.groupby(['game_id','white_first_name']).white_result.sum()
black_result = game_info.groupby(['game_id','black_first_name']).black_result.sum()

### Q.12. List All Perfect Tournament with Winner Name.

Chess Funda: Perfect Tournament means a player has won all the matches excluding draw matches. e.g Player A has won 5 matches out of 7
Matches in tournament where 2 matches are draw and player B has won 0 matches)
Result attributes: winner_name, tournament_name

In [33]:
winners_per_tournament.sort_values('tournament_name', ascending=False)[['winner', 'tournament_name']]
winners_per_tournament.head()

,tournament_name,winner,score
0,FideChamp1993,"Karpov, Anatoly",6
4,FideChamp1996,"Karpov, Anatoly",6
12,FideChamp1998,"Anand, Viswanathan",10
97,FideChamp1999,"Khalifman, Alexander",12
142,FideChamp2000,"Anand,V",8


### Q.13. Player with highest winning ratio.
Hint: Winning ratio: (Number of rounds won)/(Number of rounds played)
Result attributes: player_name
Final result will have only one row

In [34]:
game_info["w_full_name"] = game_info.white_first_name + ' ' + game_info.w_last_name
game_info['b_full_name'] = game_info.black_first_name + ' ' + game_info.b_last_name

In [35]:
number_of_rounds_played_by_white = pd.DataFrame(game_info.groupby('w_full_name').w_full_name.count()).rename(columns={'w_full_name':'count'}).reset_index().rename(columns={'w_full_name':'player_name'})
number_of_rounds_played_by_black = pd.DataFrame(game_info.groupby('b_full_name').b_full_name.count()).rename(columns={'b_full_name':'count'}).reset_index().rename(columns={'b_full_name':'player_name'})
# number_of_players_per_player = number_of_rounds_played_by_white.add(number_of_rounds_played_by_black)
# number_of_players_per_player
s1 = number_of_rounds_played_by_black.set_index('player_name')['count']
s2 = number_of_rounds_played_by_white.set_index('player_name')['count']
number_of_players_per_player = s1.add(s2, fill_value=0).reset_index(name='count')
number_of_players_per_player.head()

,player_name,count
0,Abulhul T,2.0
1,Acs P,10.0
2,Adams Michael,49.0
3,Adams Mi,68.0
4,Adianto Utut,6.0


**Names need to be cleaned.**

In [36]:
# number of wins
number_wins = pd.DataFrame(game_info[game_info.winner !='draw'].groupby(['winner']).winner.count()).rename(columns={'winner':'count'}).reset_index().rename(columns={'winner':'player_name'})
# number_wins.player_name = number_wins.player_name.apply(lambda x: x.split(',')[0])
number_wins['first_name'] = number_wins.player_name.apply(lambda x: x.split(',')[0])
# number_wins['last_name'] = number_wins.player_name.apply(lambda x: x.split(',')[1])
number_wins.head()


,player_name,count,first_name
0,"Acs,P",3,Acs
1,"Adams, Michael",16,Adams
2,"Adams,Mi",20,Adams
3,"Adianto, Utut",2,Adianto
4,"Adianto,U",3,Adianto


### Q.14. Player who had given checkmate with Pawn.
Note: Consider all events for this query
Result attributes: player_name
Final result will have only one row

In [86]:
g=df.groupby(['piece','is_check_mate'])['player']
g

In [87]:
s=g.max()
df2=pd.DataFrame(s.reset_index())
df2.head()

,piece,is_check_mate,player
0,B,0.0,Zvjaginsev
1,K,0.0,Zvjaginsev
2,N,0.0,Zvjaginsev
3,P,0.0,Zvjaginsev
4,P,1.0,Andersson


In [88]:
d=df2.loc[(df2.piece=='P')&(df2.is_check_mate==1.0)]
d

,piece,is_check_mate,player
4,P,1.0,Andersson


### Q.15. List games where player has won game without queen.
Result attributes: game_id, event, player_name

In [92]:
winners_per_tournament = pd.DataFrame(df.groupby(['tournament_name','winner','piece']).winner.count()).rename(columns={'winner':'score'}).reset_index()
winners_per_tournament = winners_per_tournament[(winners_per_tournament.piece != 'Q') & (winners_per_tournament.winner != 'draw')] 
winners_per_tournament = winners_per_tournament.loc[winners_per_tournament.groupby(['tournament_name'])['score'].idxmax()]
winners_per_tournament.head()

,tournament_name,winner,piece,score
3,FideChamp1993,"Karpov, Anatoly",P,145
27,FideChamp1996,"Karpov, Anatoly",P,149
75,FideChamp1998,"Anand, Viswanathan",P,234
585,FideChamp1999,"Khalifman, Alexander",P,347
1125,FideChamp2000,"Shirov,A",P,180
